<a href="https://colab.research.google.com/github/jarekwan/jarwan_projekt/blob/main/PR_KONC_BOOTCAMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Przygotowanie środowiska i danych
Najpierw ustalasz, z jakich źródeł danych korzystasz — z pliku z Kaggle (Supplier Performance Dataset) oraz z danych syntetycznych wygenerowanych samodzielnie w Pythonie.
Sprawdzasz, czy dane mają spójne kolumny (np. czas dostawy, niezawodność, koszt, liczba reklamacji).
Dane z obu źródeł można analizować osobno albo połączyć w jedną tabelę.


In [ ]:
import pandas as pd
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Wczytanie danych z pliku (Supplier Performance Dataset)
Tk().withdraw()
print("Wybierz plik z danymi (Supplier Performance Dataset):")
file_path = askopenfilename()
kaggle_data = pd.read_csv(file_path)

print("Dane z Kaggle wczytane. Wymiary:", kaggle_data.shape)
display(kaggle_data.head())

# Wygenerowanie syntetycznych danych w Pythonie
synthetic_data = pd.DataFrame({
    'supplier_id': [f'S{i}' for i in range(1, 21)],
    'delivery_time': np.random.normal(5, 1.5, 20),
    'reliability': np.random.uniform(0.7, 1.0, 20),
    'cost': np.random.normal(100, 15, 20),
    'complaints': np.random.randint(0, 5, 20)
})

print("\nDane syntetyczne wygenerowane. Wymiary:", synthetic_data.shape)
display(synthetic_data.head())

# Sprawdzenie, czy dane mają spójne kolumny
print("\nKolumny w danych z Kaggle:")
print(list(kaggle_data.columns))
print("\nKolumny w danych syntetycznych:")
print(list(synthetic_data.columns))

# Dane z obu źródeł można analizować osobno albo połączyć w jedną tabelę



2️⃣ Wstępna analiza danych (EDA)

Na tym etapie poznajesz dane:
• sprawdzasz, jakie są typy kolumn i czy są braki,
• liczysz podstawowe statystyki (średnie, odchylenia),
• szukasz wartości odstających,
• oglądasz rozkłady i zależności między zmiennymi,
• analizujesz korelacje, żeby zobaczyć, które cechy są do siebie podobne lub powielają tę samą informację.
Celem EDA jest zrozumienie struktury danych i pierwsze przeczucie, jakie grupy mogą się w nich kryć.

In [ ]:
# 2️⃣ Wstępna analiza danych (EDA)

import matplotlib.pyplot as plt
import seaborn as sns

# Wybór, które dane analizujemy (można zmienić na synthetic_data)
df = kaggle_data.copy()

# Podgląd danych
print("Podgląd danych:")
display(df.head())

# Sprawdzenie typów danych i braków
print("\nTypy danych:")
print(df.dtypes)
print("\nLiczba brakujących wartości w kolumnach:")
print(df.isnull().sum())

# Statystyki opisowe
print("\nPodstawowe statystyki:")
display(df.describe())

# Wykresy rozkładów dla zmiennych numerycznych
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols].hist(figsize=(12, 8), bins=20)
plt.suptitle("Rozkłady zmiennych numerycznych", fontsize=14)
plt.show()

# Boxploty dla wykrycia wartości odstających
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[num_cols])
plt.title("Boxplot – wykrywanie wartości odstających")
plt.xticks(rotation=45)
plt.show()

# Heatmapa korelacji
plt.figure(figsize=(10, 6))
sns.heatmap(df[num_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Macierz korelacji cech")
plt.show()

# Analiza – identyfikacja najbardziej skorelowanych zmiennych
corr_matrix = df[num_cols].corr().abs()
high_corr = corr_matrix.unstack().sort_values(ascending=False)
high_corr = high_corr[high_corr < 1].head(5)
print("\nNajsilniejsze korelacje między zmiennymi:")
print(high_corr)


3️⃣ Przygotowanie danych do analizy
Na tym etapie porządkujesz dane tak, żeby nadawały się do modelowania:
•	uzupełniasz lub usuwasz brakujące wartości,
•	standaryzujesz wszystkie cechy (np. żeby koszt i niezawodność były w tej samej skali),
•	opcjonalnie możesz usunąć kolumny, które są bardzo silnie skorelowane (bo nie wnoszą nowej informacji).
To etap, który sprawia, że PCA i klasteryzacja będą działały prawidłowo.


In [ ]:
# 3️⃣ Przygotowanie danych do analizy

from sklearn.preprocessing import StandardScaler

# Kopia danych
data_model = df.copy()

# Usuwanie kolumn nienumerycznych (jeśli występują)
num_cols = data_model.select_dtypes(include=['float64', 'int64']).columns
data_model = data_model[num_cols]

# Uzupełnianie braków medianą
data_model = data_model.fillna(data_model.median())

# Standaryzacja cech (ważne dla PCA i K-Means)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_model)
data_scaled = pd.DataFrame(data_scaled, columns=data_model.columns)

print("Dane po standaryzacji – pierwsze 5 wierszy:")
display(data_scaled.head())

# Opcjonalnie: usunięcie silnie skorelowanych kolumn (>0.9)
corr_matrix = data_scaled.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

if len(to_drop) > 0:
    print("\nUsuwam silnie skorelowane kolumny:", to_drop)
    data_scaled = data_scaled.drop(columns=to_drop)
else:
    print("\nBrak bardzo silnie skorelowanych kolumn — nic nie usuwam.")

print("\nOstateczny kształt danych po przygotowaniu:", data_scaled.shape)


4️⃣ Redukcja wymiarowości (PCA)
PCA (Principal Component Analysis) pomaga uprościć dane — czyli zmniejszyć liczbę kolumn, zachowując większość informacji.
Dzięki temu:
•	możesz łatwiej narysować dane na wykresie (np. w 2D),
•	klasteryzacja staje się prostsza i mniej podatna na szum,
•	widać, które kombinacje cech najlepiej różnicują dostawców.
Po PCA uzyskasz kilka nowych „składowych”, które zastępują oryginalne kolumny, ale wciąż dobrze opisują zróżnicowanie dostawców.


In [ ]:
# 4️⃣ Redukcja wymiarowości (PCA)

from sklearn.decomposition import PCA

# Uruchomienie PCA – najpierw bez ograniczenia liczby komponentów
pca_full = PCA()
pca_full.fit(data_scaled)

# Analiza wariancji wyjaśnianej
explained_var = np.cumsum(pca_full.explained_variance_ratio_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_var) + 1), explained_var, marker='o')
plt.title('Wykres skumulowanej wariancji wyjaśnianej przez PCA')
plt.xlabel('Liczba komponentów')
plt.ylabel('Skumulowana wariancja')
plt.grid(True)
plt.show()

# Wybór liczby komponentów (np. tłumaczących 90% wariancji)
n_components = np.argmax(explained_var >= 0.9) + 1
print(f"Liczba komponentów PCA tłumaczących ≥90% wariancji: {n_components}")

# Redukcja wymiarów
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(data_scaled)

# Konwersja do DataFrame
data_pca = pd.DataFrame(data_pca, columns=[f"PC{i+1}" for i in range(n_components)])
print("\nDane po redukcji PCA – pierwsze 5 wierszy:")
display(data_pca.head())

# Wizualizacja w 2D (pierwsze 2 komponenty)
if n_components >= 2:
    plt.figure(figsize=(7, 6))
    plt.scatter(data_pca['PC1'], data_pca['PC2'], s=30, alpha=0.7)
    plt.title('Wizualizacja danych po PCA (2 pierwsze komponenty)')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.show()


5️⃣ Klasteryzacja — K-Means
K-Means to metoda, która dzieli dane na określoną liczbę grup.
Najpierw sprawdzasz, ile klastrów ma sens — testujesz różne liczby i obserwujesz wykres metody Elbow (gdzie krzywa przestaje gwałtownie spadać).
Następnie oceniasz jakość grup za pomocą wskaźnika Silhouette Score, który mówi, jak dobrze punkty pasują do swojej grupy.
Po wybraniu najlepszego modelu opisujesz charakterystykę każdej grupy (np. tanio i szybko, drogo i solidnie, tanio ale niestabilnie itd.).


In [ ]:
# 5️⃣ Klasteryzacja — K-Means

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Sprawdzenie optymalnej liczby klastrów metodą Elbow
wcss = []  # Within Cluster Sum of Squares
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(data_pca)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, wcss, marker='o')
plt.title('Metoda Elbow – wybór liczby klastrów K')
plt.xlabel('Liczba klastrów (k)')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

# Ocena jakości grup metodą Silhouette
silhouette_scores = []
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(data_pca)
    score = silhouette_score(data_pca, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(8, 5))
plt.plot(k_range, silhouette_scores, marker='o', color='orange')
plt.title('Silhouette Score dla różnych k')
plt.xlabel('Liczba klastrów (k)')
plt.ylabel('Średni Silhouette Score')
plt.grid(True)
plt.show()

# Wybór najlepszego K (maksymalny Silhouette)
best_k = k_range[np.argmax(silhouette_scores)]
print(f"Najlepsza liczba klastrów wg Silhouette Score: {best_k}")

# Finalny model K-Means
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
data_pca['Cluster_KMeans'] = kmeans_final.fit_predict(data_pca)

# Podgląd rozkładu klastrów
print("\nLiczba obserwacji w każdym klastrze:")
print(data_pca['Cluster_KMeans'].value_counts())

# Wizualizacja klastrów w 2D (po PCA)
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x='PC1', y='PC2',
    hue='Cluster_KMeans',
    palette='tab10',
    data=data_pca,
    s=50, alpha=0.8
)
plt.title('Wizualizacja klastrów K-Means (po PCA)')
plt.legend(title='Cluster')
plt.show()


6️⃣ Klasteryzacja — DBSCAN
DBSCAN działa inaczej — nie wymaga podania liczby grup.
Sam wykrywa obszary, gdzie punkty są gęsto skupione, i traktuje je jako klastry, a punkty odległe uznaje za „odstające”.
Ta metoda jest dobra, jeśli dane są nieregularne lub zawierają pojedynczych, nietypowych dostawców.
Porównujesz wyniki DBSCAN z K-Means, by zobaczyć, czy ta metoda lepiej wychwytuje wyjątki lub bardziej naturalne grupy.


In [ ]:
# 6️⃣ Klasteryzacja — DBSCAN

from sklearn.cluster import DBSCAN

# Test kilku wartości parametrów eps i min_samples
eps_values = [0.3, 0.5, 0.7, 1.0]
min_samples_values = [3, 5, 10]

best_dbscan = None
best_silhouette = -1
best_params = None

for eps in eps_values:
    for min_s in min_samples_values:
        db = DBSCAN(eps=eps, min_samples=min_s)
        labels = db.fit_predict(data_pca.iloc[:, :-1])  # pomijamy kolumnę z klastrem K-Means

        # Pomijamy przypadki, gdzie wszystkie punkty są uznane za -1 (szum)
        if len(set(labels)) > 1 and len(set(labels)) < len(data_pca):
            score = silhouette_score(data_pca.iloc[:, :-1], labels)
            if score > best_silhouette:
                best_silhouette = score
                best_dbscan = db
                best_params = (eps, min_s)

if best_dbscan:
    print(f"Najlepsze parametry DBSCAN: eps={best_params[0]}, min_samples={best_params[1]}")
    print(f"Silhouette Score: {best_silhouette:.3f}")

    data_pca['Cluster_DBSCAN'] = best_dbscan.fit_predict(data_pca.iloc[:, :-2])
else:
    print("Nie udało się znaleźć sensownych klastrów DBSCAN (zbyt rozproszone dane).")

# Sprawdzenie liczby grup
if 'Cluster_DBSCAN' in data_pca.columns:
    print("\nLiczba punktów w każdej grupie (DBSCAN):")
    print(data_pca['Cluster_DBSCAN'].value_counts())

    # Wizualizacja klastrów DBSCAN
    plt.figure(figsize=(8, 6))
    sns.scatterplot(
        x='PC1', y='PC2',
        hue='Cluster_DBSCAN',
        palette='tab10',
        data=data_pca,
        s=50, alpha=0.8
    )
    plt.title('Wizualizacja klastrów DBSCAN (po PCA)')
    plt.legend(title='Cluster')
    plt.show()


7️⃣ Ocena i wizualizacja wyników
Na tym etapie porównujesz oba podejścia (K-Means i DBSCAN).
Tworzysz wizualizacje po PCA, żeby zobaczyć klastry w układzie 2D lub 3D.
Analizujesz, jak średnie wartości cech różnią się między grupami — które grupy mają najniższy koszt, które najwięcej reklamacji itp.
Interpretujesz wyniki w kontekście biznesowym: co oznaczają poszczególne grupy i jak mogą pomóc w zarządzaniu dostawcami.


In [ ]:
# 7️⃣ Ocena i wizualizacja wyników

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score

# --- 7.0 Zbiór pomocniczy: standaryzowane cechy jako DataFrame (jeśli jeszcze go nie masz)
# Załóżmy, że masz: X_scaled (np.array) i num_cols (lista cech numerycznych)
df_scaled = pd.DataFrame(X_scaled, columns=num_cols, index=df_clean.index)

# --- 7.1 Porównanie jakości: Silhouette (K-Means vs DBSCAN) – jeśli masz zapisane wartości, pokażmy je
try:
    print(f"[INFO] K-Means: k={best_k} | silhouette={best_kmeans_sil:.3f}")
except:
    print("[INFO] Brak zapisanych metryk dla K-Means (best_k / best_kmeans_sil).")

if 'Cluster_DBSCAN' in data_pca.columns:
    labels_db = data_pca['Cluster_DBSCAN'].values
    # Silhouette liczymy tylko gdy jest co najmniej 2 klastry i nie wszystkie to szum (-1)
    if len(set(labels_db)) > 1 and not (set(labels_db) == {-1}):
        sil_db = silhouette_score(data_pca[['PC1', 'PC2']], labels_db)
        print(f"[INFO] DBSCAN: clusters={len(set(labels_db))} | silhouette={sil_db:.3f}")
    else:
        print("[INFO] DBSCAN: brak sensownych klastrów do obliczenia silhouette.")
else:
    print("[INFO] Brak kolumny 'Cluster_DBSCAN' w data_pca.")

# --- 7.2 Wizualizacje 2D po PCA dla obu metod
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.scatterplot(
    data=data_pca, x='PC1', y='PC2', hue='Cluster_KMeans',
    palette='tab10', s=50, ax=axes[0], alpha=0.85
)
axes[0].set_title('K-Means clusters (PCA 2D)')
axes[0].legend(title='Cluster', bbox_to_anchor=(1.02, 1), loc='upper left')

if 'Cluster_DBSCAN' in data_pca.columns:
    sns.scatterplot(
        data=data_pca, x='PC1', y='PC2', hue='Cluster_DBSCAN',
        palette='tab10', s=50, ax=axes[1], alpha=0.85
    )
    axes[1].set_title('DBSCAN clusters (PCA 2D)')
    axes[1].legend(title='Cluster', bbox_to_anchor=(1.02, 1), loc='upper left')
else:
    axes[1].axis('off')
    axes[1].set_title('DBSCAN: no labels')

plt.tight_layout()
plt.show()

# --- 7.3 Charakterystyka klastrów: średnie cech (w skali standaryzowanej)
def cluster_profile_table(labels_col_name):
    """
    Buduje tabelę średnich cech (z-score) per klaster oraz liczności.
    labels_col_name: 'Cluster_KMeans' lub 'Cluster_DBSCAN'
    """
    if labels_col_name not in data_pca.columns:
        return None

    labels = data_pca[labels_col_name]
    tmp = df_scaled.copy()
    tmp[labels_col_name] = labels.values  # dopasowanie indeksu

    # Usuwamy szum DBSCAN (-1) z profilu, ale pokazujemy liczność
    if labels_col_name == 'Cluster_DBSCAN':
        counts = tmp[labels_col_name].value_counts().sort_index()
        prof = tmp[tmp[labels_col_name] != -1].groupby(labels_col_name)[num_cols].mean().round(2)
    else:
        counts = tmp[labels_col_name].value_counts().sort_index()
        prof = tmp.groupby(labels_col_name)[num_cols].mean().round(2)

    return prof, counts

# K-Means profile
km_prof, km_counts = cluster_profile_table('Cluster_KMeans')
print("\n[INFO] K-Means — liczność klastrów:")
print(km_counts)
print("\n[INFO] K-Means — profil (średnie z-score cech):")
display(km_prof)

# DBSCAN profile (jeśli są etykiety)
if 'Cluster_DBSCAN' in data_pca.columns:
    db_prof, db_counts = cluster_profile_table('Cluster_DBSCAN')
    print("\n[INFO] DBSCAN — liczność klastrów (uwzględnia szum -1):")
    print(db_counts)
    if db_prof is not None and not db_prof.empty:
        print("\n[INFO] DBSCAN — profil (średnie z-score cech, bez szumu -1):")
        display(db_prof)

# --- 7.4 Heatmapy profili cech per klaster (ułatwia „czytanie” grup)
def plot_cluster_heatmap(profile_df, title):
    if profile_df is None or profile_df.empty:
        print(f"[WARN] {title}: brak danych do heatmapy.")
        return
    plt.figure(figsize=(min(12, 1.2*len(profile_df.columns)), 0.7*len(profile_df)))
    sns.heatmap(profile_df, cmap='vlag', center=0, annot=True, fmt='.2f')
    plt.title(title)
    plt.xlabel('Features (z-score)')
    plt.ylabel('Cluster')
    plt.tight_layout()
    plt.show()

plot_cluster_heatmap(km_prof, 'K-Means: cluster profile (z-score means)')
if 'Cluster_DBSCAN' in data_pca.columns and db_prof is not None and not db_prof.empty:
    plot_cluster_heatmap(db_prof, 'DBSCAN: cluster profile (z-score means, noise removed)')

# --- 7.5 Krótka, automatyczna etykietyzacja klastrów (przykład)
# Heurystyka: wskaż cechy o najwyższym bezwzględnym z-score w danym klastrze
def auto_labels_from_profile(profile_df, top_n=2):
    if profile_df is None or profile_df.empty:
        return {}
    labels = {}
    for c in profile_df.index:
        top_feats = profile_df.loc[c].abs().sort_values(ascending=False).head(top_n).index.tolist()
        sign = ['+' if profile_df.loc[c, f] > 0 else '-' for f in top_feats]
        pretty = ', '.join([f"{f}({s})" for f, s in zip(top_feats, sign)])
        labels[c] = pretty
    return labels

km_labels = auto_labels_from_profile(km_prof, top_n=2)
print("\n[INFO] Propozycje podpisów K-Means (na podstawie z-score):")
for c, lab in km_labels.items():
    print(f"  • Cluster {c}: {lab}")

if 'Cluster_DBSCAN' in data_pca.columns and db_prof is not None and not db_prof.empty:
    db_labels = auto_labels_from_profile(db_prof, top_n=2)
    print("\n[INFO] Propozycje podpisów DBSCAN (bez -1):")
    for c, lab in db_labels.items():
        print(f"  • Cluster {c}: {lab}")

# --- 7.6 (Opcja) Rzutowanie wyników z powrotem do oryginalnego df, by łatwo filtrować dostawców
df_results = df_clean.copy()
df_results[['PC1','PC2']] = data_pca[['PC1','PC2']]
df_results['Cluster_KMeans'] = data_pca['Cluster_KMeans'].values
if 'Cluster_DBSCAN' in data_pca.columns:
    df_results['Cluster_DBSCAN'] = data_pca['Cluster_DBSCAN'].values

# Przykładowe „fiszki” biznesowe: top 10 dostawców z klastra o najniższym średnim lead time (K-Means)
if km_prof is not None:
    try:
        lead_col_guess = [c for c in num_cols if 'lead' in c.lower() or 'delivery_time' in c.lower() or 'on_time' in c.lower()]
        if lead_col_guess:
            # zakładamy, że mniejsza wartość = lepiej (krótszy lead time)
            best_cluster = km_prof[lead_col_guess].mean(axis=1).sort_values().index[0]
            print(f"\n[INFO] K-Means: klaster o najlepszym (najniższym) średnim LEAD TIME to: {best_cluster}")
            sample = df_results[df_results['Cluster_KMeans'] == best_cluster].head(10)
            display(sample)
    except Exception as e:
        print(f"[WARN] Nie udało się wygenerować 'fiszek' biznesowych: {e}")


8️⃣ Wnioski i interpretacja biznesowa
Na końcu opisujesz, co udało się odkryć:
•	jakie grupy dostawców istnieją,
•	które są strategiczne, a które ryzykowne,
•	jak firma mogłaby wykorzystać te informacje (np. renegocjacje kontraktów, zmiana priorytetów dostaw, lepsze planowanie zapasów).
Celem jest pokazanie, że analiza ma realne zastosowanie, a nie tylko techniczny wynik
